In [2]:
%matplotlib inline
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# For data preprocessing
from tensorflow.keras.utils import to_categorical

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [3]:
df = pd.read_csv('final_data.csv')
info = pd.read_csv('info.csv')

In [29]:
info.groupby('position').count()

,Unnamed: 0,summoner_a,summoner_b,win_rate,lane_kill_rate,kda,kill_participate,damage
position,,,,,,,,
0,2141,2141,2141,2141,2141,2141,2141,2141
1,1453,1453,1453,1453,0,1453,1453,1453
2,2148,2148,2148,2148,2148,2148,2148,2148
4,1446,1446,1446,1446,1446,1446,1446,0


In [16]:
df

,top_win_rate,top_lane_kill_rate,top_kda,top_kill_participate,top_damage,jg_win_rate,jg_kda,jg_kill_participate,jg_damage,mid_win_rate,...,adc_win_rate,adc_lane_kill_rate,adc_kda,adc_kill_participate,adc_damage,sup_win_rate,sup_lane_kill_rate,sup_kda,sup_kill_participate,result
0,0.5165,0.5350,0.571429,0.529851,0.549520,0.5069,0.485660,0.489531,0.509276,0.5049,...,0.4960,0.5816,0.513333,0.509784,0.471470,0.5351,0.4985,0.436229,0.491745,1.0
1,0.4714,0.3487,0.412346,0.453865,0.452230,0.4895,0.532632,0.514573,0.510004,0.4417,...,0.5135,0.4670,0.507605,0.500000,0.539221,0.4857,0.5645,0.551515,0.512987,1.0
2,0.4194,0.5431,0.507246,0.521191,0.513936,0.5064,0.474645,0.526022,0.606890,0.4816,...,0.5248,0.4753,0.513684,0.503891,0.458632,0.5009,0.4878,0.451220,0.470395,1.0
3,0.5190,0.5149,0.500000,0.502481,0.556929,0.4197,0.441296,0.496496,0.430142,0.4943,...,0.5266,0.5789,0.508539,0.487562,0.496394,0.4602,0.5369,0.428037,0.520788,1.0
4,0.4881,0.5154,0.413146,0.467259,0.528237,0.4782,0.498069,0.505174,0.454369,0.5183,...,0.5049,0.5293,0.496109,0.509146,0.442267,0.5073,0.4807,0.507853,0.493318,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18277,0.4355,0.6879,0.508861,0.490074,0.490373,0.4798,0.509690,0.509728,0.423939,0.4874,...,0.5062,0.4594,0.486381,0.489712,0.462690,0.4563,0.5780,0.420345,0.443067,1.0
18278,0.5347,0.5320,0.470423,0.495856,0.540506,0.5029,0.519157,0.513983,0.562910,0.4780,...,0.4737,0.4553,0.496842,0.507692,0.547123,0.5026,0.6534,0.465753,0.449532,0.0
18279,0.3684,0.3871,0.334056,0.469333,0.405582,0.4860,0.502879,0.506616,0.456211,0.4889,...,0.5180,0.6372,0.576238,0.530550,0.503203,0.5084,0.5130,0.518591,0.522404,0.0
18280,0.5982,0.7423,0.588785,0.516005,0.438540,0.5061,0.537190,0.518443,0.541791,0.4082,...,0.5026,0.4902,0.472165,0.477581,0.491471,0.5124,0.6006,0.453901,0.501152,1.0


In [34]:
x_train = df[df.columns[0:-1]]
y_train = df["result"]

In [17]:
df.groupby('result').count()

,top_win_rate,top_lane_kill_rate,top_kda,top_kill_participate,top_damage,jg_win_rate,jg_kda,jg_kill_participate,jg_damage,mid_win_rate,...,mid_damage,adc_win_rate,adc_lane_kill_rate,adc_kda,adc_kill_participate,adc_damage,sup_win_rate,sup_lane_kill_rate,sup_kda,sup_kill_participate
result,,,,,,,,,,,,,,,,,,,,,
0.0,9057,9057,9057,9057,9057,9057,9057,9057,9057,9057,...,9057,9057,9057,9057,9057,9057,9057,9057,9057,9057
1.0,9225,9225,9225,9225,9225,9225,9225,9225,9225,9225,...,9225,9225,9225,9225,9225,9225,9225,9225,9225,9225


In [7]:
y_train.shape

(10707,)

In [21]:
y_train = to_categorical(y_train, 2)

In [9]:
y_train[1003]

array([0., 1.], dtype=float32)

In [22]:
FC_layers = [Dense(units=256, input_dim=23, activation='relu'),
             Dense(units=128, activation = 'relu'),
             Dense(units=2, activation='softmax')]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
model = Sequential(FC_layers)
# model.add(Dense(87, input_dim=784, activation='relu'))
# model.add(Dense(128,activation = 'relu'))
# model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               6144      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 39,298
Trainable params: 39,298
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss='mse', optimizer=SGD(lr=0.005), metrics=['accuracy'])

In [33]:
model.fit(x_train, y_train, batch_size=50, epochs=50, validation_split = 1/6)

Train on 15235 samples, validate on 3047 samples
Epoch 1/50
15235/15235 [==============================] - 3s 170us/sample - loss: 0.2496 - acc: 0.5155 - val_loss: 0.2504 - val_acc: 0.4962
Epoch 2/50
15235/15235 [==============================] - 2s 100us/sample - loss: 0.2497 - acc: 0.5131 - val_loss: 0.2504 - val_acc: 0.4949
Epoch 3/50
15235/15235 [==============================] - 1s 95us/sample - loss: 0.2496 - acc: 0.5092 - val_loss: 0.2502 - val_acc: 0.5018
Epoch 4/50
15235/15235 [==============================] - 1s 98us/sample - loss: 0.2497 - acc: 0.5126 - val_loss: 0.2502 - val_acc: 0.5018
Epoch 5/50
15235/15235 [==============================] - 1s 92us/sample - loss: 0.2497 - acc: 0.5120 - val_loss: 0.2502 - val_acc: 0.4995
Epoch 6/50
15235/15235 [==============================] - 1s 93us/sample - loss: 0.2496 - acc: 0.5120 - val_loss: 0.2503 - val_acc: 0.4959
Epoch 7/50
15235/15235 [==============================] - 2s 103us/sample - loss: 0.2496 - acc: 0.5193 - val_loss: 

In [14]:
# model.save('model1.h5')

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [36]:
X = df[df.columns[0:-1]]
Y = df["result"]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)

In [ ]:
scorel =[]
for i in range(0,500,10):
    rfc = RandomForestClassifier(n_estimators=i+1,
                                    n_jobs=-1,
                                    random_state=10
                                )
    score = cross_val_score(rfc, x_train, y_train, cv=10).mean()
    scorel.append(score)
print(max(scorel),(scorel.index(max(scorel))*10)+1)
plt.plot(range(1,501,10), scorel)

In [37]:
rfc = RandomForestClassifier(n_estimators=10,n_jobs=-1,random_state=10)
rfc = rfc.fit(x_train,y_train)
train_score = rfc.score(x_train, y_train)
test_score = rfc.score(x_test, y_test)
print('R^2 Training Score:{}'.format(train_score),'R^2 Validation Score:{}'.format(test_score))

# print()
# rfc.feature_importances_

rfc_result = pd.concat((pd.DataFrame(X.columns, columns = ['Variable']), 
                pd.DataFrame(rfc.feature_importances_.T, columns = ['Importance'])), 
                axis = 1).sort_values(by='Importance', ascending = False)

rfc_result.reset_index().drop(["index"],axis=1)

R^2 Training Score:0.9864856168350602 R^2 Validation Score:0.5038279256288735


,Variable,Importance
0,top_win_rate,0.047013
1,mid_kill_participate,0.046412
2,mid_win_rate,0.046094
3,mid_kda,0.045782
4,top_kill_participate,0.045490
5,mid_damage,0.045343
6,mid_lane_kill_rate,0.045275
7,sup_win_rate,0.045273
8,top_lane_kill_rate,0.044880
9,sup_kill_participate,0.044622
